In [1]:
"""
社畜丼toot分析NB
神野さん作のSQLiteDBファイル(https://1drv.ms/u/s!AgFTqyL78d4kc_Aj5ShUchymJBQ)を
使用しています
"""

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
%matplotlib inline
from bokeh.plotting import figure, output_notebook, show
output_notebook()

#import matplotlib as mpl
#print([f.name for f in mpl.font_manager.createFontList(mpl.font_manager.findSystemFonts())])
sns.set_style("whitegrid", {'grid.linestyle': '--'})
#sns.set(font="DejaVu Sans")
#sns.set(font="Noto Sans CJK")
sns.set(font="IPAGothic")

Loading BokehJS ...

In [2]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from sqlalchemy.ext.automap import automap_base

dbfile = 'tootData.db'
tablename = 'timelines'

def bind(dbfile, echo=True):
    class Toot(object):
        __tablename__ = tablename
    
    engine = create_engine(f'sqlite:///{dbfile}', echo=echo)
    metadata = MetaData(bind=engine)
    metadata.reflect(engine)
    from sqlalchemy.orm import mapper
    mapper(Toot, metadata.tables[tablename])
    return engine, metadata, Toot

try:
    engine, metadata, Toot = bind(dbfile, False)
except Exception:
    # http://docs.sqlalchemy.org/en/rel_1_0/faq/ormconfiguration.html#how-do-i-map-a-table-that-has-no-primary-key
    engine = create_engine(f'sqlite:///{dbfile}', echo=True)
    engine.execute(f"CREATE TABLE tmp(id INTEGER UNIQUE ON CONFLICT REPLACE PRIMARY KEY, username TEXT, toot TEXT, created_at TEXT);")
    engine.execute(f"INSERT INTO tmp SELECT * FROM {tablename};")
    engine.execute(f"drop table {tablename};")
    engine.execute(f"ALTER TABLE tmp RENAME TO {tablename};")
    engine.execute("VACUUM;")
    engine, metadata, Toot = bind(dbfile, True)

import emoji
username_table = pd.read_csv("username_table.tsv", header=None, delimiter='\t')
username_dict = dict(username_table.to_dict('split')['data'])
for username in username_dict.keys():
    if pd.isnull(username_dict[username]):
        username_dict[username] = username
    username_dict[username] = emoji.emojize(username_dict[username], use_aliases=True)

In [3]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

def row_as_dict(r):
    from sqlalchemy import inspect
    return {c.key: getattr(r, c.key)
            for c in inspect(r).mapper.column_attrs}
def toots_as_df(toots):
    return pd.DataFrame.from_records(map(row_as_dict, toots))

In [4]:
from sqlalchemy import and_, or_, not_

all_toots = toots_as_df(session.query(Toot).all())
oppai_toots = toots_as_df(session.query(Toot).filter(
    or_(
        Toot.toot.like("%おっぱい%"),
        and_(Toot.toot.like("%乳%"), not_(or_(Toot.toot.like("%牛乳%"), Toot.toot.like("豆乳")))),
        Toot.toot.like("%π%"),
        Toot.toot.like("%バスト%"),
        Toot.toot.like("%oppai%"),
        and_(Toot.toot.like("%ぱい"), not_(Toot.toot.like("%いっぱい%")))
    )
))
notkws = ["マーク", "スマート", "アーマードコア", "コンシューマー", "マーチ", "クレーマー", "グラマー", "マーライオン", "マーゲイ",
         "ジャマー", "マーカー", "マーケ", "マージン", "オマージュ", "ママー", "アーマー", "サマー", "マージ", "ハマー", "ハンマー",
         "マーメイド", "マーブル", "グンマー", "ウマー", "ゲーマー", "カスタマー", "コマーシャル", "タイマー", "マート", "クマー",
         "マーン", "マーガリン", "バルマー", "ファーマー", "テルマー", "マーマイト", 
         ]
mer_toots = toots_as_df(session.query(Toot).filter(Toot.toot.like("%マー%"))
                        .filter(not_(or_(*[Toot.toot.like(f'%{keyword}%') for keyword in notkws]))))

toots = {'総トゥート': all_toots, 'ぱいトゥート': oppai_toots, 'マートゥート': mer_toots}
for _, df in toots.items():
    df['created_at'] = pd.to_datetime(df['created_at'])

In [5]:
from ipywidgets import interact
h = 25
df = oppai_toots
#@interact(n=(0, len(df.index)//h))
#    return df[n*h:(n+1)*h]

In [6]:
from bokeh.palettes import Dark2_5 as palette
from ipywidgets import SelectionSlider

def plot_genre_toots(timespan):
    p = figure(plot_width=800, plot_height=250, x_axis_type="datetime", title="ジャンル別発言数")
    for i, (label, df) in enumerate(toots.items()):
        group = df.set_index('created_at').groupby(pd.TimeGrouper(timespan))['id'].count()
        p.line(group.keys(), group.values, alpha=0.5, line_color=palette[i], legend=label)
    show(p)

timespanSlider = SelectionSlider(
    options=['5min', '10min', '30min', '1H', '1H', '2H', '6H', '12H', 'D'],
    value='30min',
    description='graph timespan',
    continuous_update=True)
interact(plot_genre_toots, timespan=timespanSlider)

<function __main__.plot_genre_toots>

In [7]:
def plot_relative_genre_toots(timespan):
    p = figure(plot_width=800, plot_height=250, x_axis_type="datetime", title="全トゥート数で正規化したジャンル別発言数")
    for i, (label, df) in enumerate({'ぱいトゥート': oppai_toots, 'マートゥート': mer_toots}.items()):
        dividend = df.set_index('created_at').groupby(pd.TimeGrouper(timespan))['id'].count()
        divisor = all_toots.set_index('created_at').groupby(pd.TimeGrouper(timespan))['id'].count()
        group = dividend / divisor
        p.line(group.keys(), group.values, alpha=0.5, line_color=palette[i], legend=label)
    show(p)
timespanSlider = SelectionSlider(
    options=['5min', '10min', '30min', '1H', '1H', '2H', '6H', '12H', 'D'],
    value='30min',
    description='graph timespan',
    continuous_update=True)
interact(plot_relative_genre_toots, timespan=timespanSlider)

<function __main__.plot_relative_genre_toots>

In [8]:
toot_num = all_toots.groupby('username')['toot'].count().rename(username_dict).sort_values(ascending=False)
xmax = toot_num.values[0] + 500

from ipywidgets import IntSlider

@interact(n=IntSlider(min=1,max=500,value=1))
def plot_ranking(n):
    start, end = n, n+35-1
    plt.rcParams["figure.figsize"] = (12, 8)
    toot_ranged = toot_num[start-1:end]
    plt.barh(range(start, end+1), toot_ranged.values, left=0, tick_label=toot_ranged.keys())
    plt.gca().set_xlim([0, xmax])
    plt.gca().invert_yaxis()
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,y1-1, y2+1))
    plt.show()



In [9]:
import emoji
print(emoji.emojize('Python is :thumbsup:', use_aliases=True))

Python is 👍
